In [21]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristics OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [22]:
#decide which df to use
df = pd.read_csv("Dataframes/analysts.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]
X_var_names = ["Earnings Per Share – Coefficient of Variation", "Number of Analysts", "Recommendation - Mean (1-5)", "Recommendation change", "Price Target - Mean", "Price Target - Standard Deviation"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Recommendation change,Number of Analysts,Recommendation - Mean (1-5),Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,3.625620,0.00000,7.0,3.00000,36.00000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.046169,-0.12500,7.0,2.87500,42.57143,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.551601,0.00000,8.0,2.87500,45.00000,4.30946
3,AVY.N,2013-10-01,Materials,8.095,2.775994,-0.25000,7.0,2.62500,47.71429,3.45230
4,AVY.N,2014-01-01,Materials,1.471,3.204412,-0.29167,8.0,2.33333,54.62500,3.42555
...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,2.836295,0.00000,8.0,2.30000,545.28571,50.48823
20116,POOL.OQ,2022-01-01,Retailing,40.267,3.269867,0.00000,8.0,2.30000,571.00000,41.33833
20117,POOL.OQ,2022-04-01,Retailing,34.342,6.630736,-0.30000,9.0,2.00000,519.88889,37.49206
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.625249,0.20000,9.0,2.20000,435.37500,57.57373


#### OLS Regression

In [23]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()


In [24]:
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Recommendation change,Number of Analysts,Recommendation - Mean (1-5),Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,3.625620,0.00000,7.0,3.00000,36.00000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.046169,-0.12500,7.0,2.87500,42.57143,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.551601,0.00000,8.0,2.87500,45.00000,4.30946
3,AVY.N,2013-10-01,Materials,8.095,2.775994,-0.25000,7.0,2.62500,47.71429,3.45230
4,AVY.N,2014-01-01,Materials,1.471,3.204412,-0.29167,8.0,2.33333,54.62500,3.42555
...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,2.836295,0.00000,8.0,2.30000,545.28571,50.48823
20116,POOL.OQ,2022-01-01,Retailing,40.267,3.269867,0.00000,8.0,2.30000,571.00000,41.33833
20117,POOL.OQ,2022-04-01,Retailing,34.342,6.630736,-0.30000,9.0,2.00000,519.88889,37.49206
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.625249,0.20000,9.0,2.20000,435.37500,57.57373


In [25]:
y = df_accuracy_new["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = df_accuracy_new[X_var_names]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.006
Model:                                              OLS   Adj. R-squared:                  0.006
Method:                                   Least Squares   F-statistic:                     18.38
Date:                                  Sun, 26 Feb 2023   Prob (F-statistic):           2.11e-21
Time:                                          18:50:17   Log-Likelihood:                -75239.
No. Observations:                                 17707   AIC:                         1.505e+05
Df Residuals:                                     17700   BIC:                         1.505e+05
Df Model:                                             6                                         
Covariance Type:                              nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                             8.1019      0.939      8.625      0.000       6.261       9.943
Earnings Per Share – Coefficient of Variation     0.0644      0.008      8.124      0.000       0.049       0.080
Number of Analysts                                0.0306      0.018      1.700      0.089      -0.005       0.066
Recommendation - Mean (1-5)                      -0.9476      0.350     -2.705      0.007      -1.634      -0.261
Recommendation change                            -4.3816      1.029     -4.259      0.000      -6.398      -2.365
Price Target - Mean                              -0.0012      0.001     -0.902      0.367      -0.004       0.001
Price Target - Standard Deviation                 0.0237      0.011      2.084      0.037       0.001       0.046
==============================================================================
Omnibus:                     3507.141   Durbin-Watson:                   1.641
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34939.729
Skew:                           0.665   Prob(JB):                         0.00
Kurtosis:                       9.752   Cond. No.                     2.18e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### OLS Pooled Regression

In [26]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True)
df_accuracy_new.dtypes

Instrument                                               object
Date                                             datetime64[ns]
GICS Industry Group Name                                 object
Earnings Per Share - Actual Surprise                    float64
Earnings Per Share – Coefficient of Variation           float64
Recommendation change                                   float64
Number of Analysts                                      float64
Recommendation - Mean (1-5)                             float64
Price Target - Mean                                     float64
Price Target - Standard Deviation                       float64
dtype: object

In [27]:
#df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share – Coefficient of Variation,Recommendation change,Number of Analysts,Recommendation - Mean (1-5),Price Target - Mean,Price Target - Standard Deviation
0,AVY.N,2013-01-01,Materials,11.178,3.625620,0.00000,7.0,3.00000,36.00000,2.44949
1,AVY.N,2013-04-01,Materials,2.482,2.046169,-0.12500,7.0,2.87500,42.57143,3.92272
2,AVY.N,2013-07-01,Materials,1.068,1.551601,0.00000,8.0,2.87500,45.00000,4.30946
3,AVY.N,2013-10-01,Materials,8.095,2.775994,-0.25000,7.0,2.62500,47.71429,3.45230
4,AVY.N,2014-01-01,Materials,1.471,3.204412,-0.29167,8.0,2.33333,54.62500,3.42555
...,...,...,...,...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,2.836295,0.00000,8.0,2.30000,545.28571,50.48823
20116,POOL.OQ,2022-01-01,Retailing,40.267,3.269867,0.00000,8.0,2.30000,571.00000,41.33833
20117,POOL.OQ,2022-04-01,Retailing,34.342,6.630736,-0.30000,9.0,2.00000,519.88889,37.49206
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.625249,0.20000,9.0,2.20000,435.37500,57.57373


# with grouping


In [28]:
from statsmodels.iolib.summary2 import summary_col
# Split the DataFrame into groups based on the stocks
df = df_accuracy_new[df_accuracy_new['GICS Industry Group Name'] == 'Technology Hardware & Equipment']
groups = df.groupby('Instrument')

# Define a function to perform OLS regression on each group
def ols_regression(group):
    # Define the dependent and independent variables
    if len(group) < 2:
        return None
    y = group['Earnings Per Share - Actual Surprise']
    X = group[X_var_names]

    # Add a constant to the independent variables
    X = sm.add_constant(X)

    # Fit the OLS model and return the results
    model = sm.OLS(y, X).fit()
    return model

# Apply the function to each group of data
results = groups.apply(ols_regression)
results = results.dropna()
# Collect the results into a new DataFrame

# Print the coefficients, t-values, and p-values
# print('Coefficients:')
# print(coefficients)
# print('\nT-values:')
# print(t_values)
# print('\nP-values:')
models = results.tolist()
summary = summary_col(models, stars=True, float_format='%0.2f', model_names=results.index.tolist())

# Print the combined summary table
summary

,AAPL.OQ,APH.N,CSCO.OQ,FFIV.OQ,GLW.N,HPQ.N,JNPR.N,MSI.N,NTAP.OQ,STX.OQ,TDY.N,TEL.N,TRMB.OQ,WDC.OQ,ZBRA.OQ
const,-27.78,-4.87,-0.49,18.75,53.60*,-15.51,-48.58,28.03,134.92,-32.17,39.48,-12.14,-10.34,-42.20,12.39
,(35.21),(16.33),(10.38),(20.48),(31.39),(19.61),(78.10),(32.81),(85.66),(55.76),(31.25),(11.70),(35.28),(74.23),(22.28)
Earnings Per Share – Coefficient of Variation,0.57,0.52,1.01***,0.89**,0.71*,-0.45,0.00,1.07,1.28,-0.97,1.03*,2.58***,2.79***,-0.82*,-0.53
,(0.78),(0.78),(0.19),(0.34),(0.36),(0.74),(0.50),(1.26),(1.02),(1.01),(0.55),(0.43),(0.65),(0.47),(0.58)
Number of Analysts,0.16,1.51*,-0.07,-0.23,-0.45,-0.66**,0.64,-2.40*,-2.60**,-0.23,-1.54,-0.25,0.84,2.09,0.28
,(0.35),(0.84),(0.14),(0.23),(0.62),(0.27),(0.62),(1.40),(1.05),(1.29),(1.18),(0.59),(1.40),(1.30),(1.27)
Recommendation - Mean (1-5),11.01,-5.14,3.28,-2.98,-15.34,13.34*,13.43,7.62,-7.35,18.78,-8.23,8.07,-2.48,4.15,-4.24
,(14.08),(7.96),(3.63),(4.30),(9.21),(7.50),(17.47),(9.78),(17.61),(16.34),(9.57),(5.36),(12.57),(20.37),(6.34)
Recommendation change,-25.76,0.43,1.83,11.40*,-8.09,-15.57,5.76,0.77,-27.16,-7.29,0.76,-8.03,2.55,-0.95,-7.76
,(16.32),(11.01),(4.59),(6.44),(9.41),(10.07),(16.02),(18.57),(18.32),(22.46),(14.24),(7.26),(14.61),(26.79),(8.97)
